In [ ]:
import time
from itertools import islice
from itertools import groupby
from itertools import count
inputPath = '/home/bdm/BDM/per-vehicle-records-2021-01-29.csv'
outputPath = '/home/bdm/livestream/'
def split_part(t, data_size): 
    d = count()
    for _, s in groupby(t, lambda _: next(d) // data_size):
        yield s
def save_file(data, path):
    with open(path, 'w') as f:
        f.writelines(record + '\n' for record in data)
input = (line.rstrip('\n') for line in open(inputPath))
input = islice(input, 1, None)
input = filter(lambda x: "test" not in x.split(',')[10].lower(), input)
# spliting in part
inputpart =split_part(input, 10)
read = 0
#stores part as separate files (countdata1,countdata2, countdata3, etc.) in the streaming directory
for part in inputpart:
    read += 1
    save_file(part, '{}/countdata{}.csv'.format(outputPath, read))
    time.sleep(5)

In [1]:
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
from pyspark.sql import Row
import time


def Stream1(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, classname=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="q1", keyspace="streams")\
        .save(mode="append")
    except:
        pass





ssc = StreamingContext(sc, 5)
lines = ssc.textFileStream("/home/bdm/livestream/")
records=lines.map(lambda line:line.split(","))
q1=records.map(lambda x: ((x[14]),1))
q1=q1.reduceByKey(lambda a,b : a+b)
q1.foreachRDD(Stream1)
q1.pprint()

ssc.start()
time.sleep(100)
ssc.stop(stopSparkContext=False)

-------------------------------------------
Time: 2022-05-04 02:38:50
-------------------------------------------

-------------------------------------------
Time: 2022-05-04 02:38:55
-------------------------------------------
('"CAR"', 6)
('"HGV_ART"', 2)
('"LGV"', 1)
('"CARAVAN"', 1)

-------------------------------------------
Time: 2022-05-04 02:39:00
-------------------------------------------
('"HGV_ART"', 2)
('"HGV_RIG"', 2)
('"CARAVAN"', 1)
('"CAR"', 4)
('"LGV"', 1)

-------------------------------------------
Time: 2022-05-04 02:39:05
-------------------------------------------
('"CAR"', 5)
('"HGV_RIG"', 2)
('"HGV_ART"', 2)
('"LGV"', 1)

-------------------------------------------
Time: 2022-05-04 02:39:10
-------------------------------------------
('"CAR"', 5)
('"HGV_ART"', 2)
('"LGV"', 3)

-------------------------------------------
Time: 2022-05-04 02:39:15
-------------------------------------------
('"CAR"', 5)
('"HGV_RIG"', 1)
('"HGV_ART"', 4)

-----------------------

In [2]:
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
from pyspark.sql import Row
import time


def Stream2(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, classname=row[0],avg_speed=row[1],)))
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="q2", keyspace="streams")\
        .save(mode="append")
    except:
        pass





ssc = StreamingContext(sc, 5)
lines = ssc.textFileStream("/home/bdm/livestream/")
records=lines.map(lambda line:line.split(","))
q2=records.map(lambda x: (x[14].strip('"'), float(x[18].strip('"')))).groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sum(x[1]) / len(x[1])))
q2.foreachRDD(Stream2)
q2.pprint()

ssc.start()
time.sleep(100)
ssc.stop(stopSparkContext=False)

-------------------------------------------
Time: 2022-05-04 02:40:35
-------------------------------------------
('HGV_ART', 92.5)
('CARAVAN', 87.0)
('LGV', 96.0)
('CAR', 119.0)

-------------------------------------------
Time: 2022-05-04 02:40:40
-------------------------------------------
('HGV_ART', 88.77777777777777)
('CAR', 84.0)

-------------------------------------------
Time: 2022-05-04 02:40:45
-------------------------------------------
('HGV_ART', 86.0)
('LGV', 125.33333333333333)
('CAR', 110.0)

-------------------------------------------
Time: 2022-05-04 02:40:50
-------------------------------------------
('CAR', 94.875)
('HGV_ART', 84.5)

-------------------------------------------
Time: 2022-05-04 02:40:55
-------------------------------------------
('CAR', 100.5)
('HGV_ART', 88.0)
('CARAVAN', 88.0)
('LGV', 108.66666666666667)

-------------------------------------------
Time: 2022-05-04 02:41:00
-------------------------------------------
('BUS', 62.5)
('LGV', 100.0

In [3]:
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
from pyspark.sql import Row
import time


def Stream3(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, site=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="q3", keyspace="streams")\
        .save(mode="append")
    except:
        pass

ssc = StreamingContext(sc, 5)
lines = ssc.textFileStream("/home/bdm/livestream/")
records=lines.map(lambda line:line.split(","))
q3=records.map(lambda x: ((x[0]),1))
q3=q3.reduceByKey(lambda x,y: x+y )
q3=q3.transform(
    lambda rdd:rdd.sortBy(lambda x:x[1],ascending=False)
)
q3.foreachRDD(Stream3)
q3.pprint()

ssc.start()
time.sleep(300)
ssc.stop(stopSparkContext=False)

-------------------------------------------
Time: 2022-05-04 02:42:15
-------------------------------------------
('"000000020076"', 7)
('"000000020077"', 3)

-------------------------------------------
Time: 2022-05-04 02:42:20
-------------------------------------------
('"000000020077"', 9)
('"000000020078"', 1)

-------------------------------------------
Time: 2022-05-04 02:42:25
-------------------------------------------
('"000000020078"', 8)
('"000000020079"', 2)

-------------------------------------------
Time: 2022-05-04 02:42:30
-------------------------------------------
('"000000020079"', 7)
('"000000020081"', 2)
('"000000020082"', 1)

-------------------------------------------
Time: 2022-05-04 02:42:35
-------------------------------------------
('"000000020083"', 5)
('"000000020084"', 3)
('"000000020082"', 2)

-------------------------------------------
Time: 2022-05-04 02:42:40
-------------------------------------------
('"000000020085"', 7)
('"000000020084"', 3)

--

In [4]:
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
from pyspark.sql import Row
import time


def Stream4(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, m50parts=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="q4", keyspace="streams")\
        .save(mode="append")
    except:
        pass

ssc = StreamingContext(sc, 5)
lines = ssc.textFileStream("/home/bdm/livestream/")
records=lines.map(lambda line:line.split(","))
q4=records.filter(lambda x: "HGV" in x[14])
q4=records.map(lambda x: (x[10].strip('"'), 1))
q4=q4.reduceByKey(lambda x, y: x + y)
q4.foreachRDD(Stream4)
q4.pprint()

ssc.start()
time.sleep(100)
ssc.stop(stopSparkContext=False)

-------------------------------------------
Time: 2022-05-04 02:47:15
-------------------------------------------

-------------------------------------------
Time: 2022-05-04 02:47:20
-------------------------------------------
('Northbound 2 (fast)', 4)
('Southbound 1 (slow)', 3)
('Southbound 2 (fast)', 3)

-------------------------------------------
Time: 2022-05-04 02:47:25
-------------------------------------------
('Southbound 2 (fast)', 2)
('Southbound 1 (slow)', 3)
('Northbound 1 (slow)', 4)
('Northbound 2 (fast)', 1)

-------------------------------------------
Time: 2022-05-04 02:47:30
-------------------------------------------
('Southbound 2 (fast)', 2)
('Southbound 1 (slow)', 3)
('Northbound 1 (slow)', 3)
('Northbound 2 (fast)', 2)

-------------------------------------------
Time: 2022-05-04 02:47:35
-------------------------------------------
('Southbound 1 (slow)', 5)
('Southbound 2 (fast)', 3)
('Northbound 2 (fast)', 1)
('Northbound 1 (slow)', 1)

--------------------